# Data Preperation

In [1]:
import pandas as pd
import numpy as np

## Making the first,second,third and fourth columns of the dataframe as date,month,year and time

In [2]:
data=pd.DataFrame()
temp=pd.read_csv('../Data Sources/ENTSO-E/Load/Croatia.csv')
data['Day']=temp['Time (CET)'].str[:2]
data['Month']=temp['Time (CET)'].str[3:5]
data['Year']=temp['Time (CET)'].str[6:10]
data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]

### In the country speciic Load data set, Austria,Belgium,Germany,Hungary,Netherlands report data every 15 minutes. Therefore, these countries have 35044 data points per year. UK and Ireland report data every 30 minutes henece these countries have 17522 datapoints per year. All the others report every 1 hour hence have 8761 datapoints per year. In Genearion dataset, situation is same as abobe except Belgium reports hourly data hence have 8761 datapoints. In Transmission dataset, all countries report data hourly. Therefroe,it is easy if all the data are converted to hourly data. To do that in the countries with 35044 datapoints, mean is calculated in every successive 4 datapoints. In the countries with 17522 datapoints, mean is calculated in every successive 2 datapoints.`

In [3]:
def hourly_data(df):
    length=len(df.index)
    if length==35044:
        divider=4
    elif length==17522:
        divider=2
    else:
        divider=1

    # Following command creates a numpy array with a length similar to the length of the dataframe. Values of the array are obtained by getting the floor division of the length value. For example, when divider=4, this array will be [0,0,0,0,1,1,1,1,2,....]. Then the rows of the dataframe will be grouped according to the order of the numpy array with the mean value of those 4 rows. For example, in the numpy array first 4 values are similar. Accordingly first 4 rows of the dataframe will be grouped and get the mean value of those rows 
    
    df=df.groupby(np.arange(length)//divider).mean()
    return(df)    

# Preparing Load Data

In [4]:
def load(countries):

    load_data=pd.DataFrame()
    load_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    # In the following command we open the csv file of each country and save the data in the 'temp' dataframe. Then we call the 'hourly_data' function to make all the time steps to hourly data. Then the column 'Actual Total Load [MW] - {country_name} ({country_code})' is saved in the new dataframe 'load_data' under the column name '{country_code}'. For example, the column 'Actual Total Load [MW] - Germany (DE)' in the 'temp' dataframe will be saved in the 'load_data' dataframe under the column name 'DE'.
    
    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Load/{country}.csv')
        temp=hourly_data(temp)
        load_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']
        
    return(load_data)

# Preparing Generation Data

In [5]:
def generation(countries):
    
    generation_data=pd.DataFrame()
    generation_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    # In the following command we open the csv file of each country and save the data in the 'temp' dataframe. Then we call the 'hourly_data' function to make all the time steps to hourly data. Then we get the column names of the 'temp' dataframe into a numpy array called 'fuels'. Then we get the column name without the '- Actual Aggregated [MW]' part into a new variable called 'edited_fuel'. Then the column 'fuel' is saved in the new dataframe 'load_data' under the column name '{country_code} - {edited_fuel}'. For example, the column 'Biomass  - Actual Aggregated [MW]' of Germany in the 'temp' dataframe will be saved in the 'load_data' dataframe under the column name 'DE - Biomass'. Then we filter the column heads which have the string '{country_code}' in the column name and get the sum of each row in those columns and save it in a new column called '{country_code} - Total'. For example, 'DE - Total', 'UK - Total' 
        
    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/Generation/{country}.csv',low_memory=False)
        temp=hourly_data(temp)

        fuels=np.array(temp.columns)
        for fuel in fuels:
            edited_fuel=fuel[:-26]
            generation_data[f'{abbr} - {edited_fuel}']=temp[fuel]
        generation_data[f'{abbr} - Total'] = generation_data.filter(like=abbr).sum(axis=1)

    return(generation_data)